In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import numpy as np
from scipy.signal import find_peaks


In [ ]:
sampling_rate=2*1e6
sample_time=1/sampling_rate
N=2000000
def ou_noise(n, dt=1.0, mu=0.0, tau=20.0, sigma=1.0, x0=0.0, rng=None):
    rng = np.random.default_rng(rng)
    x = np.empty(n)
    x[0] = x0
    a = np.exp(-dt / tau)
    sd = sigma * np.sqrt(1 - a*a)
    for t in range(1, n):
        x[t] = mu + a*(x[t-1] - mu) + sd * rng.normal()
    return x
#I am setting tau to be 50ms
x = ou_noise(N, dt=sample_time, tau=100*1e-3, sigma=0.5)
plt.plot(x)
print(10*1e-3/sample_time)

In [ ]:
Channel_dict={}
num_channels=10
actual_coh_time={}
for i in range(10):
    tau_I=random.randint(1,25)
    tau_Q=random.randint(1,25)
    Channel_dict[i]={'I':ou_noise(N,dt=sample_time,tau=tau_I*1e-3,sigma=0.5),'Q':ou_noise(N,dt=sample_time,tau=tau_Q*1e-3,sigma=0.5)}
    actual_coh_time[i]={'I':tau_I*1e-3/sample_time,'Q':tau_Q*1e-3/sample_time}
print(actual_coh_time)

In [ ]:
from numpy.fft import rfft, irfft

def autocorr_fft(x):
    x = np.asarray(x)
    x = x - x.mean()            # optional, improves robustness
    n = len(x)
    nfft = 1 << (2*n-1).bit_length()  # next pow2 for speed
    X = rfft(x, nfft)
    r = irfft(X * np.conj(X), nfft)[:n]   # non-negative lags
    r = r / (r[0] + 1e-20)      # normalize
    return r

def find_coh_time_fft(x, threshold=1e-2):
    r = autocorr_fft(x)
    # first lag where it drops below threshold
    idx = np.argmax(r <= threshold)  # returns 0 if first element meets
    if r[0] <= threshold:            # guard unlikely case
        return 0
    if idx == 0:                     # no drop found
        return len(r)                # cap at window length
    return idx

def find_coherence_time_per_channel(Wifi_symbol_dict):
    num_samples_per_coherence_detection=int(50*1e-3/sample_time) #We need 50ms data for this
    t_coh_dict={}
    for channel in Wifi_symbol_dict:
        t_coh_dict[channel]={'I':0,'Q':0}
        i=0
        number_samples=len(Wifi_symbol_dict[channel]['I'])
        t_coh_distribution_arr_I=[]
        t_coh_distribution_arr_Q=[]
        while num_samples_per_coherence_detection*i<number_samples:
            #normalised_I_samples=Wifi_symbol_dict[channel]['I']
            I_curr_samples=Wifi_symbol_dict[channel]['I'][i*num_samples_per_coherence_detection:(i+1)*num_samples_per_coherence_detection]
            Q_curr_samples=Wifi_symbol_dict[channel]['Q'][i*num_samples_per_coherence_detection:(i+1)*num_samples_per_coherence_detection]
            t_coh_distribution_arr_I.append(find_coh_time_fft(I_curr_samples))
            t_coh_distribution_arr_Q.append(find_coh_time_fft(Q_curr_samples))
            i+=1
            #print(i)
         #break
        t_coh_dict[channel]['I']=np.mean(t_coh_distribution_arr_I)
        t_coh_dict[channel]['Q']=np.mean(t_coh_distribution_arr_Q)
    
    return t_coh_dict


# #print(find_coh_time_fft(x[:100000]))
print(find_coherence_time_per_channel(Channel_dict))


In [ ]:
print(Channel_dict)

In [ ]:
# multi armed bandit to select next channel to scan
int(50*1e-3/sample_time)

In [ ]:
# each channel is one arm
num_channels = 10 # number of arms

arm_coherent_time = {}
arm_dwell_time = {}

arms_with_reward = {}
arm_last_played_time = {}
arm_coherent_time = {}

for i in range(num_channels):
    arms_with_reward[i] = 0
    arm_last_played_time[i] = 0
    arm_coherent_time[i] = min(find_coherence_time_per_channel(Channel_dict)[i]['I'], find_coherence_time_per_channel(Channel_dict)[i]['Q'])
    #arm_coherent_time[i] = random.randint(400, 1000)
    arm_dwell_time[i] = int(1e6 / arm_coherent_time[i]) # fix this too


In [ ]:
# print(Channel_dict)
# print(len(Channel_dict))
# print(Channel_dict[9])
# print(len(Channel_dict[9]['I']))
# print(arm_coherent_time[1])

In [ ]:
def get_reward(channel_chosen, current_sample_index, arm_last_played_time, arm_coherent_time):
    reward = (current_sample_index - arm_last_played_time[channel_chosen])/arm_coherent_time[channel_chosen]
    return (reward)

In [ ]:
# def selected_arm(epsilon):
#     k = random.random()
#     if (k < epsilon) : 
#         next_arm = random.randint(1, N)
#     else:
#         next_arm = max(arms_with_reward, key = arms_with_reward.get)
#     return(next_arm)

def selected_arm():
    next_arm = max(arms_with_reward, key = arms_with_reward.get)
    return(next_arm)

In [ ]:
Wifi_output_samples={}
temp_output_samples = {}
for channel in Channel_dict:
    Wifi_output_samples[channel]={'I':[],'Q':[]}  # storing in the form of a list of elements (sample_num,value) for I and Q
# epsilon = 1.0
# epsilon_decay = 0.995
total_samples = 2000000

current_sample_index = 0
while(current_sample_index<total_samples):

    temp_output_samples = {}

    for i in range(num_channels):
        arms_with_reward[i] = get_reward(i, current_sample_index, arm_last_played_time, arm_coherent_time)
    next_arm = selected_arm()
    temp_output_samples[next_arm] = {}
    temp_output_samples[next_arm]['I'] = []
    temp_output_samples[next_arm]['Q'] = []
    arms_with_reward[next_arm] = 0
    for i in range(arm_dwell_time[next_arm]):
        if current_sample_index==total_samples-1:
            current_sample_index+=1
            break
        Wifi_output_samples[next_arm]['I'].append([current_sample_index, Channel_dict[next_arm]['I'][current_sample_index]])
        Wifi_output_samples[next_arm]['Q'].append([current_sample_index, Channel_dict[next_arm]['Q'][current_sample_index]])
        temp_output_samples[next_arm]['I'].append(Channel_dict[next_arm]['I'][current_sample_index])
        temp_output_samples[next_arm]['Q'].append(Channel_dict[next_arm]['Q'][current_sample_index])
        current_sample_index+=1   

    # print(temp_output_samples)

    coh_time_to_update = find_coherence_time_per_channel(temp_output_samples)
    arm_coherent_time[i] = min(coh_time_to_update[next_arm]['I'], coh_time_to_update[next_arm]['Q'])

    arm_last_played_time[next_arm] = current_sample_index

In [ ]:
#print(arm_stddev)
#print(arm_mean_time)
print(Wifi_output_samples)

In [ ]:
# def interpolate_smartly(data):
#     "The input data is of the form [index,value] we need to be able to diffrentiate between periodic and non periodic signals to do this we are just linearly interpolating in stft"
#     output_wifi_samples={}
#     for channel in data:
        
#         output_wifi_samples[channel]={'I':np.array([0 for i in range(2000000)]),'Q':np.array([0 for i in range(2000000)])}
#         I = np.zeros(2_000_000, dtype=np.float64)  # force float64
#         for cur_sample, val in data[channel]['I']:
#             idx = int(cur_sample)
#             I[idx] = float(val)
#         output_wifi_samples[channel]['I'] = I  # ensure the mutated array is stored

#         Q = np.zeros(2_000_000, dtype=np.float64)  # force float64
#         for cur_sample, val in data[channel]['Q']:
#             idx = int(cur_sample)
#             Q[idx] = float(val)
#         output_wifi_samples[channel]['Q'] = Q  # ensure the mutated array is stored

#     return output_wifi_samples


# Wifi_output_data_after_interpolation=interpolate_smartly(Wifi_output_samples)

# Per_channel_time_frequency_spectrum={}

# for channel in Wifi_symbols:
#     Per_channel_time_frequency_spectrum[channel]={}
#     I_f_t_spectrum=[]
#     Q_f_t_spectrum=[]
#     for i in range(num_fft-1):
#         #print((i+1)*num_samples_per_fft)
#         I_f_t_spectrum.append(abs(np.fft.fft(Wifi_output_data_after_interpolation[channel]['I'][i*num_samples_per_fft:(i+1)*num_samples_per_fft]))[:num_samples_per_fft//2])
#         Q_f_t_spectrum.append(abs(np.fft.fft(Wifi_output_data_after_interpolation[channel]['Q'][i*num_samples_per_fft:(i+1)*num_samples_per_fft]))[:num_samples_per_fft//2])
#     Per_channel_time_frequency_spectraum[channel]['I']=np.array(I_f_t_spectrum)
#     Per_channel_time_frequency_spectrum[channel]['Q']=np.array(Q_f_t_spectrum)



In [ ]:
#plt.plot(Wifi_output_data_after_interpolation[1]['I'])
#print(Wifi_output_data_after_interpolation[1]['I'][0])

In [ ]:
# fig,ax=plot_fft_rows_colormap_freq_on_y(Per_channel_time_frequency_spectrum[2]['I'])
# plt.show()

Doing Naive method

In [ ]:
Wifi_naive_symbols={}
size_of_naive_samples_per_channel=100000

cur_num=0
for channel in Wifi_symbols:
    Wifi_naive_symbols[channel]={'I':[],'Q':[]}

while cur_num<total_samples:
    for channel in Wifi_symbols:
        for i in range(size_of_naive_samples_per_channel):
            Wifi_naive_symbols[channel]['I'].append((cur_num,Wifi_symbols[channel]['I'][cur_num]))
            Wifi_naive_symbols[channel]['Q'].append((cur_num,Wifi_symbols[channel]['Q'][cur_num]))
            cur_num+=1
        #cur_num+=size_of_naive_samples_per_channel
        if cur_num>=total_samples:
            break
#I = np.zeros(2_000_000, dtype=np.float64)  # force float64
print(Wifi_naive_symbols[1]['I'][:100])
#Wifi_naive_symbols[channel]['I'] = I  # ensure the mutated array is stored


In [ ]:
Wifi_output_data_after_interpolation_naive=interpolate_smartly(Wifi_naive_symbols)

Per_channel_time_frequency_spectrum={}

for channel in Wifi_symbols:
    Per_channel_time_frequency_spectrum[channel]={}
    I_f_t_spectrum=[]
    Q_f_t_spectrum=[]
    for i in range(num_fft-1):
        #print((i+1)*num_samples_per_fft)
        I_f_t_spectrum.append(abs(np.fft.fft(Wifi_output_data_after_interpolation_naive[channel]['I'][i*num_samples_per_fft:(i+1)*num_samples_per_fft]))[:num_samples_per_fft//2])
        Q_f_t_spectrum.append(abs(np.fft.fft(Wifi_output_data_after_interpolation_naive[channel]['Q'][i*num_samples_per_fft:(i+1)*num_samples_per_fft]))[:num_samples_per_fft//2])
    Per_channel_time_frequency_spectrum[channel]['I']=np.array(I_f_t_spectrum)
    Per_channel_time_frequency_spectrum[channel]['Q']=np.array(Q_f_t_spectrum)


fig,ax=plot_fft_rows_colormap_freq_on_y(Per_channel_time_frequency_spectrum[2]['I'])
plt.show()